In [21]:
# import library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim

In [4]:
data = pd.read_excel('data/all_states_all_years.xlsx')
data.head()

,Year,Date_Temp,Date,State,Region,Place,Name
0,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak, Johor"
1,2015.0,1/1,1/1/2015,Johor,Tangkak,Kg. Sialang Dalam,"Kg. Sialang Dalam, Tangkak, Johor"
2,2015.0,3/1,3/1/2015,Johor,Kota Tinggi,Kg. Lepau,"Kg. Lepau, Kota Tinggi, Johor"
3,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seri Makmor,"Kg. Seri Makmor, Tangkak, Johor"
4,2015.0,12/8,12/8/2015,Johor,Tangkak,Kg. Seberang Gajah,"Kg. Seberang Gajah, Tangkak, Johor"


In [26]:
# testing geolocator library
geolocator = Nominatim(user_agent="u2004763@siswa.um.edu.my")
location = geolocator.geocode("Johor Bahru") # testing

print(location.point)
print(location.address)

# print out longitude and latitude
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Johor+Bahru&format=json&limit=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)')))

In [10]:
# geocoding function
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

data[['Latitude', 'Longitude']] = data.apply(lambda x: my_geocoder(x['Name']), axis=1)

# check the percentage of data successfully geocoded
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

97.94238683127571% of addresses were geocoded!


In [11]:
# Drop location that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))
data.crs = {'init': 'epsg:4326'}
data.head()

c:\Users\Khor Kean Teng\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Year,State,Place,Name,Latitude,Longitude,geometry
0,2015,Kelantan,Kuala Krai,"Kuala Krai, Kelantan",5.424618,102.152251,POINT (102.15225 5.42462)
1,2015,Kelantan,Manek Urai,"Manek Urai, Kelantan",5.388385,102.233477,POINT (102.23348 5.38838)
2,2015,Sabah,Beaufort,"Beaufort, Sabah",5.345809,115.743874,POINT (115.74387 5.34581)
3,2015,Sabah,Tenom,"Tenom, Sabah",4.910403,115.905095,POINT (115.90510 4.91040)
4,2015,Sabah,Membakut,"Membakut, Sabah",5.475288,115.803470,POINT (115.80347 5.47529)


In [12]:
# export csv
data.to_csv('data/Flood Data Updated Geocoded.csv', index=False)